# Выгрузка данных

In [23]:
import yfinance as yf

# getting data from Yahoo Finance
stock_name1 = 'IBM'  
data1 = yf.download(stock_name1, start="1970-01-01", end="2023-05-10")

stock_name2 = 'XOM' 
data2 = yf.download(stock_name2, start="2000-01-01", end="2023-05-10")

stock_name3 = 'TSLA' 
data3 = yf.download(stock_name3, start="2015-01-01", end="2023-05-10")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Подключение библиотек и создание модели

In [11]:
# import plotly package for graphs
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [20]:
def lstm(stock_name, data, batch_size, epochs):
    # Choose only Close price of stock
    data = data.filter(['Close'])
    dataset = data.values
    # Train data - 80%, test - 20%
    training_data_len = int(np.ceil( len(dataset) * .80 ))
    
    # Scale our data from 0 to 1
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)
    
    # Use our scaled data for training
    train_data = scaled_data[0:int(training_data_len), :]
    x_train = []
    y_train = []

    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    
    # Build LSTM model
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape = (x_train.shape[1], 1)))
    model.add(Dropout(0.35))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(25, activation = 'relu'))
    model.add(Dense(1))
    
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    # Тrain the model
    model.fit(x_train, y_train, batch_size, epochs)
    
    # Structure of the model
    keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

    # Create test dataset
    test_data = scaled_data[training_data_len - 60: , :]
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])

    x_test = np.array(x_test)

    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

    # Predict on test data
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)

    # For finding error we use RMSE formula, but MSE can be used too
    # report performance
    mse = mean_squared_error(y_test, predictions)
    print('MSE: '+str(mse))
    mae = mean_absolute_error(y_test, predictions)
    print('MAE: '+str(mae))
    rmse = math.sqrt(mean_squared_error(y_test, predictions))
    print('RMSE: '+str(rmse))
    mape = np.mean(np.abs(predictions - y_test)/np.abs(y_test))
    print('MAPE: '+str(mape))
    
    # Graphs
    train = data[:training_data_len]
    valid = data[training_data_len:]
    #valid['Predictions'] = predictions
    train_gr = np.reshape(train, (len(train),))
    train_gr = train_gr['Close']
    valid_gr = np.reshape(valid, (len(valid),))
    valid_gr = valid_gr['Close']
    preds_gr = np.reshape(predictions, (len(predictions),))
    

    x_train = list(range(0, len(train_data)))
    x_valid = list(range(len(train_data)-1, len(dataset)))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_train, y=train_gr, mode='lines+markers', marker=dict(size=4),  name='train', marker_color='#39304A'))
    fig.add_trace(go.Scatter(x=x_valid, y=valid_gr, mode='lines+markers', marker=dict(size=4), name='valid', marker_color='blue'))
    fig.add_trace(go.Scatter(x=x_valid, y=preds_gr, mode='lines+markers', marker=dict(size=4), name='predictions', marker_color='red'))
    fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  plot_bgcolor='#FFFFFF',  
                  xaxis=dict(gridcolor = 'lightgrey'),
                  yaxis=dict(gridcolor = 'lightgrey'),    
                  title_text = f'{stock_name} цены на акции', title_x = 0.5,
                  xaxis_title="Дата",
                  yaxis_title="Цены на акции",
                  margin=dict(l=0, r=0, t=30, b=0))
    fig.show()
    
    # Predict stock prices for next moth
    data_new = yf.download(stock_name, start="2000-01-01", end="2023-01-01")

    data_new = data_new.filter(['Close'])
    dataset = data_new.values
    training_data_len = len(dataset)

    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)

    test_data = scaled_data[training_data_len - len(data_new): , :]
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(20, len(test_data)):
        x_test.append(test_data[i-20:i, 0])

    x_test = np.array(x_test)

    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

    hist_data_new = yf.download(stock_name, start="2010-01-01", end="2023-05-01")
    hist_data_new = hist_data_new.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
    hist_data_new = hist_data_new['Close']
    hist_data_new = np.array(hist_data_new)
    pred_lstm = model.predict(x_test)
    pred_lstm = pred_lstm[:-1]
    pred_lstm = scaler.inverse_transform(pred_lstm)

    
    # build graphs
    preds_gr = np.reshape(pred_lstm, (len(pred_lstm),))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(0, 21)), y=hist_data_new, mode='lines+markers',  name='исторические', marker_color='#39304A'))
    fig.add_trace(go.Scatter(x=list(range(0, 21)), y=preds_gr, mode='lines+markers', name='предсказанные', marker_color='red'))
    fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  plot_bgcolor='#FFFFFF',  
                  xaxis=dict(gridcolor = 'lightgrey'),
                  yaxis=dict(gridcolor = 'lightgrey'),    
                  title_text = f'{stock_name} предсказание', title_x = 0.5,
                  xaxis_title="Дата",
                  yaxis_title="Цены на акции",
                  margin=dict(l=0, r=0, t=30, b=0))
    fig.show()
    
    return pred_lstm, rmse

In [17]:
lstm_pred, lstm_rmse = lstm(stock_name1, data1, 10, 21)

Epoch 1/21
163/163 [==============================] - 21s 96ms/step - loss: 0.0156 - accuracy: 0.0012
Epoch 2/21
163/163 [==============================] - 16s 97ms/step - loss: 0.0059 - accuracy: 0.0012
Epoch 3/21
163/163 [==============================] - 15s 91ms/step - loss: 0.0044 - accuracy: 0.0012
Epoch 4/21
163/163 [==============================] - 15s 91ms/step - loss: 0.0037 - accuracy: 0.0012
Epoch 5/21
163/163 [==============================] - 15s 91ms/step - loss: 0.0032 - accuracy: 0.0012
Epoch 6/21
163/163 [==============================] - 16s 96ms/step - loss: 0.0028 - accuracy: 0.0012
Epoch 7/21
163/163 [==============================] - 15s 92ms/step - loss: 0.0025 - accuracy: 0.0012
Epoch 8/21
163/163 [==============================] - 15s 92ms/step - loss: 0.0023 - accuracy: 0.0012
Epoch 9/21
163/163 [==============================] - 15s 91ms/step - loss: 0.0021 - accuracy: 0.0012
Epoch 10/21
163/163 [==============================] - 15s 91ms/step - loss: 0.001

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
181/181 [==============================] - 4s 15ms/step


In [15]:
# Check size of predictions array
print(lstm_pred.shape)

(22, 1)


In [21]:
lstm_pred2, lstm_rmse2 = lstm(stock_name2, data2, 10, 21)

Epoch 1/21
464/464 [==============================] - 38s 74ms/step - loss: 0.0049 - accuracy: 2.1552e-04
Epoch 2/21
464/464 [==============================] - 40s 87ms/step - loss: 0.0017 - accuracy: 2.1552e-04
Epoch 3/21
464/464 [==============================] - 33s 72ms/step - loss: 0.0012 - accuracy: 2.1552e-04
Epoch 4/21
464/464 [==============================] - 34s 73ms/step - loss: 8.7731e-04 - accuracy: 2.1552e-04
Epoch 5/21
464/464 [==============================] - 35s 76ms/step - loss: 7.6794e-04 - accuracy: 2.1552e-04
Epoch 6/21
464/464 [==============================] - 33s 72ms/step - loss: 7.0340e-04 - accuracy: 2.1552e-04
Epoch 7/21
464/464 [==============================] - 35s 75ms/step - loss: 7.0189e-04 - accuracy: 2.1552e-04
Epoch 8/21
464/464 [==============================] - 34s 74ms/step - loss: 6.5628e-04 - accuracy: 2.1552e-04
Epoch 9/21
464/464 [==============================] - 33s 72ms/step - loss: 6.5139e-04 - accuracy: 2.1552e-04
Epoch 10/21
464/464 [=

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
181/181 [==============================] - 3s 15ms/step


In [24]:
lstm_pred3, lstm_rmse3 = lstm(stock_name3, data3, 10, 21)

Epoch 1/21
163/163 [==============================] - 21s 95ms/step - loss: 0.0031 - accuracy: 6.1652e-04
Epoch 2/21
163/163 [==============================] - 16s 95ms/step - loss: 8.8621e-04 - accuracy: 6.1652e-04
Epoch 3/21
163/163 [==============================] - 15s 92ms/step - loss: 9.2832e-04 - accuracy: 6.1652e-04
Epoch 4/21
163/163 [==============================] - 15s 92ms/step - loss: 7.2231e-04 - accuracy: 6.1652e-04
Epoch 5/21
163/163 [==============================] - 15s 92ms/step - loss: 7.3644e-04 - accuracy: 6.1652e-04
Epoch 6/21
163/163 [==============================] - 16s 98ms/step - loss: 7.5381e-04 - accuracy: 6.1652e-04
Epoch 7/21
163/163 [==============================] - 16s 96ms/step - loss: 5.6368e-04 - accuracy: 6.1652e-04
Epoch 8/21
163/163 [==============================] - 15s 94ms/step - loss: 7.0879e-04 - accuracy: 6.1652e-04
Epoch 9/21
163/163 [==============================] - 15s 92ms/step - loss: 6.4658e-04 - accuracy: 6.1652e-04
Epoch 10/21
16

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
98/98 [==============================] - 4s 18ms/step


#Для минутных тиков

In [31]:
# getting data from Yahoo Finance
stock_name4 = 'IBM'  
data4 = yf.download(stock_name4, start="2023-05-05", end="2023-05-12", interval = "1m")

stock_name5 = 'XOM' 
data5 = yf.download(stock_name5, start="2023-05-05", end="2023-05-12", interval = "1m")

stock_name6 = 'TSLA' 
data6 = yf.download(stock_name6, start="2023-05-05", end="2023-05-12", interval = "1m")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [27]:
def lstm1(stock_name, data, batch_size, epochs):
    # Choose only Close price of stock
    data = data.filter(['Close'])
    dataset = data.values
    # Train data - 80%, test - 20%
    training_data_len = int(np.ceil( len(dataset) * .80 ))
    
    # Scale our data from 0 to 1
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)
    
    # Use our scaled data for training
    train_data = scaled_data[0:int(training_data_len), :]
    x_train = []
    y_train = []

    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    
    # Build LSTM model
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape = (x_train.shape[1], 1)))
    model.add(Dropout(0.35))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(25, activation = 'relu'))
    model.add(Dense(1))
    
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    # Тrain the model
    model.fit(x_train, y_train, batch_size, epochs)
    
    # Structure of the model
    keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

    # Create test dataset
    test_data = scaled_data[training_data_len - 60: , :]
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])

    x_test = np.array(x_test)

    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

    # Predict on test data
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)

    # For finding error we use RMSE formula, but MSE can be used too
    # report performance
    mse = mean_squared_error(y_test, predictions)
    print('MSE: '+str(mse))
    mae = mean_absolute_error(y_test, predictions)
    print('MAE: '+str(mae))
    rmse = math.sqrt(mean_squared_error(y_test, predictions))
    print('RMSE: '+str(rmse))
    mape = np.mean(np.abs(predictions - y_test)/np.abs(y_test))
    print('MAPE: '+str(mape))
    
    # Graphs
    train = data[:training_data_len]
    valid = data[training_data_len:]
    #valid['Predictions'] = predictions
    train_gr = np.reshape(train, (len(train),))
    train_gr = train_gr['Close']
    valid_gr = np.reshape(valid, (len(valid),))
    valid_gr = valid_gr['Close']
    preds_gr = np.reshape(predictions, (len(predictions),))
    

    x_train = list(range(0, len(train_data)))
    x_valid = list(range(len(train_data)-1, len(dataset)))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_train, y=train_gr, mode='lines+markers', marker=dict(size=4),  name='train', marker_color='#39304A'))
    fig.add_trace(go.Scatter(x=x_valid, y=valid_gr, mode='lines+markers', marker=dict(size=4), name='valid', marker_color='blue'))
    fig.add_trace(go.Scatter(x=x_valid, y=preds_gr, mode='lines+markers', marker=dict(size=4), name='predictions', marker_color='red'))
    fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  plot_bgcolor='#FFFFFF',  
                  xaxis=dict(gridcolor = 'lightgrey'),
                  yaxis=dict(gridcolor = 'lightgrey'),    
                  title_text = f'{stock_name} цены на акции', title_x = 0.5,
                  xaxis_title="Дата",
                  yaxis_title="Цены на акции",
                  margin=dict(l=0, r=0, t=30, b=0))
    fig.show()
    
    # Predict stock prices for next moth
    data_new = yf.download(stock_name, start="2023-05-05", end="2023-05-12", interval = "1m")

    data_new = data_new.filter(['Close'])
    dataset = data_new.values
    training_data_len = len(dataset)

    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)

    test_data = scaled_data[training_data_len - len(data_new): , :]
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(20, len(test_data)):
        x_test.append(test_data[i-20:i, 0])

    x_test = np.array(x_test)

    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

    hist_data_new = yf.download(stock_name, start="2023-05-05", end="2023-05-12", interval = "1m")
    hist_data_new = hist_data_new.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
    hist_data_new = hist_data_new['Close']
    hist_data_new = np.array(hist_data_new)
    pred_lstm = model.predict(x_test)
    pred_lstm = pred_lstm[:-1]
    pred_lstm = scaler.inverse_transform(pred_lstm)

    
    # build graphs
    preds_gr = np.reshape(pred_lstm, (len(pred_lstm),))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(0, 21)), y=hist_data_new, mode='lines+markers',  name='исторические', marker_color='#39304A'))
    fig.add_trace(go.Scatter(x=list(range(0, 21)), y=preds_gr, mode='lines+markers', name='предсказанные', marker_color='red'))
    fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  plot_bgcolor='#FFFFFF',  
                  xaxis=dict(gridcolor = 'lightgrey'),
                  yaxis=dict(gridcolor = 'lightgrey'),    
                  title_text = f'{stock_name} предсказание', title_x = 0.5,
                  xaxis_title="Дата",
                  yaxis_title="Цены на акции",
                  margin=dict(l=0, r=0, t=30, b=0))
    fig.show()
    
    return pred_lstm, rmse

In [28]:
lstm_pred4, lstm_rmse4 = lstm(stock_name4, data4, 1, 40)

Epoch 1/40
1492/1492 [==============================] - 63s 39ms/step - loss: 0.0099 - accuracy: 0.0000e+00
Epoch 2/40
1492/1492 [==============================] - 58s 39ms/step - loss: 0.0037 - accuracy: 0.0000e+00
Epoch 3/40
1492/1492 [==============================] - 73s 49ms/step - loss: 0.0030 - accuracy: 0.0000e+00
Epoch 4/40
1492/1492 [==============================] - 59s 39ms/step - loss: 0.0027 - accuracy: 0.0000e+00
Epoch 5/40
1492/1492 [==============================] - 58s 39ms/step - loss: 0.0026 - accuracy: 0.0000e+00
Epoch 6/40
1492/1492 [==============================] - 59s 39ms/step - loss: 0.0023 - accuracy: 0.0000e+00
Epoch 7/40
1492/1492 [==============================] - 58s 39ms/step - loss: 0.0022 - accuracy: 0.0000e+00
Epoch 8/40
1492/1492 [==============================] - 59s 39ms/step - loss: 0.0022 - accuracy: 0.0000e+00
Epoch 9/40
1492/1492 [==============================] - 58s 39ms/step - loss: 0.0019 - accuracy: 0.0000e+00
Epoch 10/40
1492/1492 [=====

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
181/181 [==============================] - 3s 15ms/step


In [33]:
lstm_pred5, lstm_rmse5 = lstm(stock_name5, data5, 10, 21)

Epoch 1/21
149/149 [==============================] - 17s 87ms/step - loss: 0.0240 - accuracy: 6.7204e-04
Epoch 2/21
149/149 [==============================] - 13s 88ms/step - loss: 0.0059 - accuracy: 6.7204e-04
Epoch 3/21
149/149 [==============================] - 13s 87ms/step - loss: 0.0042 - accuracy: 6.7204e-04
Epoch 4/21
149/149 [==============================] - 13s 88ms/step - loss: 0.0044 - accuracy: 6.7204e-04
Epoch 5/21
149/149 [==============================] - 13s 88ms/step - loss: 0.0035 - accuracy: 6.7204e-04
Epoch 6/21
149/149 [==============================] - 13s 88ms/step - loss: 0.0031 - accuracy: 6.7204e-04
Epoch 7/21
149/149 [==============================] - 13s 89ms/step - loss: 0.0028 - accuracy: 6.7204e-04
Epoch 8/21
149/149 [==============================] - 13s 86ms/step - loss: 0.0023 - accuracy: 6.7204e-04
Epoch 9/21
149/149 [==============================] - 13s 87ms/step - loss: 0.0019 - accuracy: 6.7204e-04
Epoch 10/21
149/149 [=========================

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
181/181 [==============================] - 3s 15ms/step


In [32]:
lstm_pred6, lstm_rmse6 = lstm(stock_name6, data6, 10, 21)

Epoch 1/21
150/150 [==============================] - 20s 94ms/step - loss: 0.0128 - accuracy: 6.7024e-04
Epoch 2/21
150/150 [==============================] - 14s 93ms/step - loss: 0.0046 - accuracy: 6.7024e-04
Epoch 3/21
150/150 [==============================] - 14s 93ms/step - loss: 0.0039 - accuracy: 6.7024e-04
Epoch 4/21
150/150 [==============================] - 14s 94ms/step - loss: 0.0033 - accuracy: 6.7024e-04
Epoch 5/21
150/150 [==============================] - 14s 94ms/step - loss: 0.0028 - accuracy: 6.7024e-04
Epoch 6/21
150/150 [==============================] - 14s 94ms/step - loss: 0.0025 - accuracy: 6.7024e-04
Epoch 7/21
150/150 [==============================] - 14s 93ms/step - loss: 0.0020 - accuracy: 6.7024e-04
Epoch 8/21
150/150 [==============================] - 14s 92ms/step - loss: 0.0018 - accuracy: 6.7024e-04
Epoch 9/21
150/150 [==============================] - 14s 95ms/step - loss: 0.0016 - accuracy: 6.7024e-04
Epoch 10/21
150/150 [=========================

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
98/98 [==============================] - 2s 16ms/step
